# EXAMPLE CODES FOR THE SIMULATED RECONSTRUCTIONS

In [ ]:
from src.forward_models.bornapprox import MVProd,MVProd_iterK
import src.optimization as opt
from src.config import RADAR_SYSTEM_CONFIGS, RSC2params
from src.nn_models.unet3D import denoiser

from src.utils.stats import psnr3D, snrdB, snrdB2nvar, awgn
from src.utils.misc import Scene, super_clear_cuda
from src.utils.data import load_dataset, load_json

import torch 
import numpy as np

Donwload the ground truth complex-valued images [1] from https://drive.google.com/drive/folders/1sxosLDMB55ZEjkti-o2d7m3V59jCAe5o?usp=sharing and place them inside "../data/datasets/IrfanC/".

[1] I. Manisali, O. Oral, and F. S. Oktem, ”Efficient physics-based learned reconstruction methods for real-time 3D near-field MIMO radar imaging”, Digital Signal Processing, vol. 144, p. 104274, 2024, doi: 10.1016/j.dsp.2023.104274.

Simulate the measurements 

In [ ]:
Nfreqs=20 # Number of frequencies used
SNR=30 # Signal to noise ratio of the simulated measurements in decibells

radar_system_params = RSC2params(RADAR_SYSTEM_CONFIGS[Nfreqs]) #get the radar system parameters
A = MVProd( freqs = radar_system_params.freqs, # frequency list
            tx_positions = radar_system_params.tx_positions, # transmitter positions
            rx_positions = radar_system_params.rx_positions, # receiver positions
            x_coords = radar_system_params.x_coords, # discretized x-coordinates
            y_coords = radar_system_params.y_coords, # discretized y-coordinates
            z_coords = radar_system_params.z_coords, # discretized z-coordinates
            path = '', save = False # you can set a path to save the forward model matrix 
            )

# load the ground truth data
x_test = load_dataset('IrfanC','test').to(dtype=torch.complex128)

# simulate the measurements
x=x_test
A=A.to('cpu')
Ax = A(x)
n_var_test = snrdB2nvar(Ax,snrdB=SNR)
N_vox=Ax.shape[-1]*Ax.shape[-2]*Ax.shape[-3]
eps = torch.sqrt(n_var_test*N_vox) # epsilon values are computed to be fed to CSALSA

torch.manual_seed(0) # set seed to add noise
y = awgn(Ax.to('cpu'),snrdB=SNR,dtype=torch.complex128) # add noise to measurements

print('Forward model matrix shape: ',A.A.shape)
print('100*(M/N) = ~{:.2f}'.format(100*A.A.shape[0]*A.A.shape[1]*A.A.shape[2]/(A.A.shape[3]*A.A.shape[4]*A.A.shape[5])))
print('y test  avg.  SNR: ',np.round(torch.mean(snrdB(Ax,y-Ax)).item(),2),'dB')
print('EPS.  test   avg.: ',np.round(torch.mean(eps).item(),2))


Set device

In [ ]:
device = torch.device("cpu")
torch.backends.cudnn.benchmark = True
if device != "cpu":
     super_clear_cuda(10)

Reconstruct the scenes:

In [ ]:
with torch.no_grad():

    model_state_dict= torch.load('trained_model/base_model.pt')

    denoiser_config=load_json('trained_model/info.json')['arch']
    denoiser_net = denoiser(**denoiser_config)
    denoiser_net.load_state_dict(model_state_dict)

    solver = opt.NNCG_CSALSA(forward_model=A, denoiser_net=denoiser_net, blind=False, noise_level_map='std', cg_max_iter=5, cg_th=1e-5, zero_phase=False, record=False)
    # you can use other denoising networks as well,
    # if you plan to use a blind denoiser set "blind" to True, 
    # if your noise level map is the variance of the noise, set "noise_level_map" to 'var'
    # cg_max_iter is the maximum number of conjugate gradient iterations
    # cg_th is the minimum relative improvement on the l2 norm before terminating the conjugate gradient iterations (cg_max_iter has priority over cg_th)
    # you should always use zero_phase=False. If true, it enforces real-positive values on reflectivity distribution, which is an incorrect assumption
    # by setting record to True, you can store the values of the data fidelity and regularization terms throughout the iterations (may be useful for debugging)


    # Other solvers:
    # solver = opt.TVCG_CSALSA(forward_model=A, chambolle_iter=5, cg_max_iter=5, cg_th=1e-5, zero_phase=False, record=False)
    # solver = opt.l1CG_CSALSA(forward_model=A, cg_max_iter=5, cg_th=1e-5, zero_phase=False, record=False)
    # solver = opt.BackProjection(radar_system_params=radar_system_params)
    # solver = opt.KirchhoffMigration(radar_system_params=radar_system_params,compansate_limited_aparture=False)
    
    solver.to(device)
    solver.clear_history()

    kwargs = {}

    # set the algorithm parameters
    start_idx=4
    batch_size=1
    target = torch.abs(x[start_idx:start_idx+batch_size,...]).to(device)
    kwargs['y'] = y[start_idx:start_idx+batch_size,...].to(device)
    kwargs['eps'] = eps[start_idx:start_idx+batch_size,...].to(device) # epsillon parameter in the paper
    kwargs['asynchronous']= True # If you are using GPU and reconstructing with batches, then you should set asynchronous to True (this allows to asynchronously stop the image updates when parallel processing, if set to False the solver will continue the optimization even after some solutions converge, this can create numerical instability)
    kwargs['tqdm_active'] = True # Set to True if you want to see an iteration counter
    kwargs['K'] = 500 # Kappa parameter in the paper
    kwargs['noise_var'] = 3.9e-2 # alpha parameter in the paper
    kwargs['max_iter']=30 # set to 10000 for TV and L1 

    # run the solver
    solver.forward(**kwargs)
    
    # get estimates (estimates are returned as normalized magnitues, you can access complex valued reconstructions by "solver.x")
    est = solver.get_estimate().cpu()

    # compute the performance
    psnrs = psnr3D(target=target,estimation=est).cpu().squeeze().tolist()
    
    # compute the average performance
    avg_psnr = np.mean(psnrs)
    print('Average PSNR (dB): ',avg_psnr)


In [ ]:
scene=Scene()
scene.show_simulated(x=est,radar_system_params=radar_system_params)